In [1]:
import urllib.request
import pandas as pd
from requests import get
from bs4 import BeautifulSoup
import gzip
import os
from time import sleep


# CSV COLUMN STRUCTURE
https://www.ncei.noaa.gov/pub/data/swdi/stormevents/csvfiles/Storm-Data-Bulk-csv-Format.pdf

In [2]:
# first, we need to get all of the available files 

url = 'https://www.ncei.noaa.gov/pub/data/swdi/stormevents/csvfiles/'
html = get(url)
soup = BeautifulSoup(html.text, 'html.parser')

# now we need to extract the file names from the parent directory
links = []
# this will grab all of the a href elements' links 
for link in [link['href'] for link in soup.find('table').find_all('a')]:
    # then check that they're a details file (the ones we actually want)
    if link[:29] == 'StormEvents_details-ftp_v1.0_':
        links.append(link)

ef_remap = {
    'F0':   'EF0',
    'F1':   'EF1',
    'EF0':  'EF0',
    'F2':   'EF2',
    'EF1':  'EF1',
    'F3':   'EF3',
    'EF2':  'EF2',
    'EFU':  'EFU',
    'F4':   'EF4',
    'EF3':  'EF3',
    'EF4':  'EF4',
    'F5':   'EF5',
    'EF5':  'EF5'
}

dtype = {
    'BEGIN_YEARMONTH': int,
    'BEGIN_DAY': int,
    'BEGIN_TIME': 'Int64',
    'END_YEARMONTH': int,
    'END_DAY': int,
    'END_TIME': int,
    'EPISODE_ID': 'Int64',
    'EVENT_ID': 'Int64',
    'STATE': str,
    'STATE_FIPS': 'Int64',
    'YEAR': 'Int64',
    'MONTH_NAME': str,
    'EVENT_TYPE': str,
    'CZ_TYPE': str,
    'CZ_FIPS': 'Int64',
    'CZ_NAME': str,
    'WFO': str,
    'BEGIN_DATE_TIME': str,
    'CZ_TIMEZONE': str,
    'END_DATE_TIME': str,
    'INJURIES_DIRECT': 'Int64',
    'INJURIES_INDIRECT': 'Int64',
    'DEATHS_DIRECT': 'Int64',
    'DEATHS_INDIRECT': 'Int64',
    'DAMAGE_PROPERTY': str,
    'DAMAGE_CROPS': str,
    'SOURCE': str,
    'MAGNITUDE': 'Float64',
    'MAGNITUDE_TYPE': str,
    'FLOOD_CAUSE': str,
    'TOR_F_SCALE': str,
    'TOR_LENGTH': 'Float64',
    'TOR_WIDTH': 'Float64',
    'TOR_OTHER_WFO': str,
    'TOR_OTHER_CZ_STATE': str,
    'TOR_OTHER_CZ_FIPS': 'Int64',
    'TOR_OTHER_CZ_NAME': str,
    'BEGIN_RANGE': 'Float64',
    'BEGIN_AZIMUTH': str,
    'BEGIN_LOCATION': str,
    'END_RANGE': 'Float64',
    'END_AZIMUTH': str,
    'END_LOCATION': str,
    'BEGIN_LAT': 'Float64',
    'BEGIN_LON': 'Float64',
    'END_LAT': 'Float64',
    'END_LON': 'Float64',
    'EPISODE_NARRATIVE': str,
    'EVENT_NARRATIVE': str,
    'DATA_SOURCE': str
}

def desuffix(r_):
    print(r_)
    if not(isinstance(r_,str)):
        return 0
    r = r_.upper()
    val = -1
    if 'K' in r:
        try:
            val = float(r[:-1]) * 1000
        except:
            val = 1000
    elif 'M' in r:
        try:
            val = float(r[:-1]) * 1000000
        except:
            val = 1000000
    elif 'B' in r:
        val = float(r[:-1]) * 1000000000
    else:
        try:
            val = float(r)
        except:
            val = 0
    return int(val)

def shout(r_):
    val = 'N/A'
    try:
        val = r_.upper()
    except:
        pass
    return val


In [ ]:
print("\n\nStarting to fetch files. This can take a couple minutes.")
out_df = None
holding = 'data'
# this needs a folder called 'data' on your local drive, so it will create one if it's not already there
if not os.path.exists(holding):
    os.makedirs(holding)
total_files = len(links)

# A lot happens here, but we start by iterating over all the files we've identified
for idx,file_path in enumerate(links):
    local_file_path = holding + '/' + file_path
    # we build the target url with the file_path we extracted earlier
    target = url + file_path
    # we go get the file in question
    urllib.request.urlretrieve(target, local_file_path)
    # it's a gzip'd file, so we need to unzip it. 
    with gzip.open(local_file_path, 'rb') as f:
        # create a df from the csv we pulled down 
        if out_df is None:
            out_df = pd.read_csv(f,dtype=dtype)
        else:
            out_df = pd.concat([out_df,pd.read_csv(f,dtype=dtype)])
    # be kind, don't pummel your free data sources with requests 
    sleep(1)
    if idx % 5 == 0:
        print(f"{idx}/{total_files} complete")
#out_df = out_df.reset_index() # this crashes the kernel sometimes, probably a memory issue 
print('\n','='*(100),'\nFetching complete! Beginning Pre-Processing\n','='*(100),'\n')

# Remap Fujita scale
# Tornado rating isn't precise, more of an comparable approximation of damage dealt
# so this conversion shouldn't affect much analysis
out_df['TOR_F_SCALE_MAPPED'] = out_df['TOR_F_SCALE'].map(ef_remap)

# Capitalize columns that contain text 
for shout_col in ['STATE','MONTH_NAME','BEGIN_AZIMUTH','BEGIN_LOCATION','END_AZIMUTH','END_LOCATION']:
    out_df[shout_col] = out_df[shout_col].apply(shout)

# Normalizing damage values
out_df['DAMAGE_CROPS_DESUFFIX'] = out_df['DAMAGE_CROPS'].apply(desuffix)
out_df['DAMAGE_PROPERTY_DESUFFIX'] = out_df['DAMAGE_PROPERTY'].apply(desuffix)

print('\n','='*(100),'\nPre-Processing complete! Reducing master file\n','='*(100),'\n')
out_df[out_df['EVENT_TYPE'].isin(['Tornado','TORNADOES'])][
    [
    'BEGIN_YEARMONTH',      # YYYYMM of event start
    'BEGIN_DAY',            # DD of event start
    'END_YEARMONTH',        # YYYYMM of event end
    'END_DAY',              # DD of event end
    'EPISODE_ID',           # NWS ID for storm
    'EVENT_ID',             # NWS ID for event (different from episode, which is mostly for storm narratives)
    'STATE',                # SPELLED OUT ALL CAPS state name
    'STATE_FIPS',           # Federal Information Processing Standard ID of the state (good for joining to other data)
    'EVENT_TYPE',           # may consider other options here for what the project is attempting to do 
    'CZ_TYPE',              # C: County/Parish, Z: NWS Public Forecast Zone, M: Marine
    'CZ_FIPS',              # See above description of FIPS but for local region ID (like Adams County)
    'CZ_NAME',              # Name of the CZ
    'INJURIES_DIRECT',      # You can probably guess 
    'INJURIES_INDIRECT',    # Difference is annoying undefined
    'DEATHS_DIRECT',        # You can probably guess, but sadder
    'DEATHS_INDIRECT',      # Same idea
    'DAMAGE_PROPERTY',      # Estimate damage to property, suffix units are all jumbled
    'TOR_F_SCALE',          # (Enhanced) Fujita Scale of the tornado 
    'TOR_LENGTH',           # Length of tornado segment on the ground (in miles, nearest tenth)
    'TOR_WIDTH',            # Width of tornado on the ground (feet)
    'BEGIN_LAT',            # Latitude of beginning of damage path
    'BEGIN_LON',            # Longitude of beginning of damage path
    'EPISODE_NARRATIVE',    # NWS' narrative of the storm that created the tornado
    'EVENT_NARRATIVE'       # NWS' narrative of the tornado itself
    ]
].to_csv('StormEvents_details_WORKING.csv')
print('\n','='*(100),'\nWorking file created! Creating Master copy.\n','='*(100),'\n')
out_df.to_csv('StormEvents_details_MASTER.csv')
print('\n','='*(100),'\nMaster copy created! Enjoy your data!\n','='*(100),'\n')
out_df = None 
